# Neural Amp Modeler ("Easy Mode" Trainer)
This notebook allows you to train a neural amp model based on a pair of input/output WAV files that you have of the amp you want to model.

**Note**:
This notebook is meant to be used on [Google Colab](https://colab.research.google.com/github/sdatkinson/neural-amp-modeler/blob/main/bin/train/easy_colab.ipynb).

🔶**Before you run**🔶

Make sure to get a GPU! (From the upper-left menu, click Runtime->Change runtime type->Select "GPU" from the "Hardware accelerator dropdown menu)

⚠**Warning**⚠

Google Colab GPU instances only last for 12 hours.
Plan your training accordingly!

## Steps:
1. Get your data
2. Installation
3. Enter metadata
4. Train!
5. Check the results and download your model

## Step 1: Get data
We're gonna need data. "Easy mode" takes out a lot of the guesswork.

### Step 1.1: Download the capture signal
"Easy mode" uses a pre-crafted "capture signal".
Download it here: [v3_0_0.wav](https://drive.google.com/file/d/1Pgf8PdE0rKB1TD4TRPKbpNo1ByR3IOm9/view?usp=drive_link).

### Step 1.2 Reamp your gear
Then reamp the gear you want to model using it. Save that reamp as "output.wav".
**Please use 48kHz, 24-bit, mono.** We'll support other sample rates etc in the future; sit tight!

### Step 1.3: upload!
Upload the input (DI) and output (amped) files you want to use by clicking the Folder icon on the left ⬅ and then clicking the upload icon.

## Step 2: Installation
Install `nam` into this Colab instance by running the next cell (hover over and click the play button that appears in the upper-left corner):

In [ ]:
!pip install neural-amp-modeler
# Hint: use the next line instead for the very latest!
# !pip install git+https://github.com/sdatkinson/neural-amp-modeler.git@main

from nam.train.colab import run as _run
run = _run
%load_ext tensorboard

## Step 3: Enter metadata
Tell us about what you're modeling!

In [ ]:
from functools import partial

import ipywidgets as widgets
from nam.models.metadata import GearType, ToneType, UserMetadata
#@markdown NAM name
name = "My model" #@param {type:"string"}
#@markdown Modeled by:
modeled_by = "Your name" #@param {type:"string"}
#@markdown Gear make:
gear_make = "GearCo" #@param {type:"string"}
#@markdown Gear model:
gear_model = "GearAmp" #@param {type:"string"}
#@markdown Gear type:
# This needs to be a literal. You need to change it by hand if you change the enum!
gear_type = "Pick from: amp, pedal, pedal_amp, amp_cab, amp_pedal_cab, preamp, studio" #@param {type:"string"}
#@markdown Tone type:
tone_type = "Pick from: clean, overdrive, crunch, hi_gain, fuzz" #@param {type:"string"}

def _verbose_enum(E, val):
    try:
        return E(val)
    except ValueError as e:
        raise ValueError(
            str(e) 
            + "\nValid choices are: " 
            + ", ".join(list(x.value for x in E))
        )

user_metadata = UserMetadata(
    name=name, 
    modeled_by=modeled_by, 
    gear_make=gear_make, 
    gear_model=gear_model, 
    gear_type=_verbose_enum(GearType, gear_type.lower()), 
    tone_type=_verbose_enum(ToneType, tone_type.lower())
)
run = partial(_run, user_metadata=user_metadata)

## Step 4: Train!
Here we go!

🕙Training will go through 100 epochs and take just over 10 minutes.🕙

If you want a better model, you can try training for more "**epochs**"--just put in a 
different number before hitting go!

"**architecture**" selects from several presets for the size of the network. This trades off
modeling quality for how expensive the resulting model will be to run.

   "**lite**" models will run approximately **1.5 times** faster than "**standard**".

   "**feather**" models will run more than **2 times** faster than "**standard**".

   "**nano**" models will run about **2.5 times** faster than "**standard**".


In [ ]:
%tensorboard --logdir /content/lightning_logs
run(
    epochs=100,
    architecture="standard",  # standard, lite, feather, nano
    fit_cab=False,  # Change me to True for full-rig modeling!
    ignore_checks=False  # Change to True to train anyways with potentially bad data
)

## Step 5: Check the results and download your model
We're done!

Have a look at the plot above to see how your model compares to the real gear you're modeling.
Hopefully it looks good!
Go to the file browser on the left panel ⬅ and download `model.nam` from the `exported_model` directory (you may need to hit the refresh button).

Additionally, if you want to continue to train this model later you can download the lightning model artifacts from `lightning_logs`. If not, that's fine too.

# 🎸 **ENJOY!** 🎸